In [1]:
## Badly titled script that will grab subjects using their metadata for randomise
import pandas as pd
import glob, os, sys
from nipype.interfaces.fsl import Merge
import shutil

metaDataFile = './hcp_subject_data_9_12_2018.csv'
nipypeResultsDir = '/data/HCP_Data/NipypeScratch/threshAndWarpPBX/'

In [2]:
subjList = [x.replace('_subject_id_','') for x in os.listdir(nipypeResultsDir)]
metadata= pd.read_csv(metaDataFile,header=0,index_col='Subject')
# metadata_group= finalmetadata.groupby(by=group)
# metadata_feature= list( metadata_group.get_group(feature)['FDT_Paths'] )

In [3]:
subjTractResults = {'M':[],'F':[]}  ### Going to create a separate one for M and Female, shuffle.. then win!

missingTractography = []
Tscale = 1000
roiName = "Hypothalamus_Bilat"

for s in subjList:
    d= os.path.join( nipypeResultsDir,'_subject_id_%s' % s,  "_roiName_%s" % roiName, '_Tscaling_%d' % Tscale, 'warp_pbxDti_to_Mni'  )
    HBF = glob.glob(d+"/Human_%s*.nii.gz"%roiName)  ### Assume and/or better only be one file

    if len(HBF)==1:
        gender = metadata.loc[int(s)]['Gender']
        subjTractResults[gender].append(HBF[0])
    else:
        missingTractography.append(s)

In [4]:
print( len(subjTractResults['F']),len(subjTractResults['M']))

fslMerger = Merge()
fslMerger.inputs.dimension = 't'
fslMerger.inputs.output_type = 'NIFTI_GZ'

subjsPerGroup = 50
outputDir = "/data/Scripts/GutmanLabNipypeWorkflows/GenderDiff_Example/"
outputDir +="50v50_20181109"


if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

### Generate Female group
fslMerger.inputs.in_files = subjTractResults['F'][:subjsPerGroup]
results = fslMerger.run()
mergedFileName = "Human_Basal_Forebrain_%s_%d.nii.gz" % ('F',subjsPerGroup)
shutil.copy(results.outputs.merged_file, os.path.join(outputDir,mergedFileName))

fslMerger.inputs.in_files = subjTractResults['M'][:subjsPerGroup]
results = fslMerger.run()
mergedFileName = "Human_Basal_Forebrain_%s_%d.nii.gz" % ('M',subjsPerGroup)
shutil.copy(results.outputs.merged_file, os.path.join(outputDir,mergedFileName))

535 464


'/data/Scripts/GutmanLabNipypeWorkflows/GenderDiff_Example/50v50_20181109/Human_Basal_Forebrain_M_50.nii.gz'

In [ ]:
### WORK ON THE VIEWER TO GENERATE THE ROI PROBABLITY MAPS FOR EACH SUBJECT

'/data/Scripts/GutmanLabNipypeWorkflows/GenderDiff_Example/50v50_20181109'

In [8]:
opd = outputDir

#"/data//Scripts/GutmanLabNipypeWorkflows/GenderDiff_Example/50v50_20181019/"

mergeFile = opd + "/HBF_50.nii.gz"
femaleFile = opd + "/Human_Basal_Forebrain_F_50.nii.gz"
maleFile = opd+ "/Human_Basal_Forebrain_M_50.nii.gz"

! fslmerge -t {mergeFile} {maleFile} {femaleFile}

In [9]:
opd = outputDir + "/"
contrastName = "HBF_50_Cohort1"

!pwd
! design_ttest2 {opd}HBF_50_Cohort1 50 50

/data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines


In [10]:
from os.path import join as oj

randoScript = """
export FSLCLUSTER_DEFAULT_QUEUE=bigmem.q
export REQUESTED_TIME=1200
randomise_parallel -i %s -o %s -d %s.mat -t %s.con -n 5000 -T -R
"""

print(randoScript )

with open(oj(opd,"randoParallelSyntax.sh"),"w") as fp:
    fp.write(randoScript%( os.path.basename(mergeFile),contrastName+"_r5000",contrastName,contrastName))
## Copy the results to the output directory


export FSLCLUSTER_DEFAULT_QUEUE=bigmem.q
export REQUESTED_TIME=1200
randomise_parallel -i %s -o %s -d %s.mat -t %s.con -n 5000 -T -R



In [ ]:
results.outputs

In [ ]:
Tscale = 5000 # 50 100 500 1000 5000
subj = subjList[20]

d= os.path.join( nipypeResultsDir,'_subject_id_%s' % subj,  '_Tscaling_%d' % Tscale, 'warp_pbxDti_to_Mni'  )
HBF = glob.glob(d+"/Human_Bas*.nii.gz")
##'Human_BasalForebrain_Left_fdt_paths_thresh_maths_wimt.nii.gz',
d,HBF